In [1]:
import pandas as pd
import numpy as np
import random
import sklearn
from copy import deepcopy

In [2]:
companies = pd.read_csv("/home/musasina/projects/teknofest/msnet/datasets/companies.csv").dropna()

In [3]:
companies = companies.convert_dtypes(convert_string=True)

In [4]:
companies_array = np.array(companies).flatten()

In [5]:
df_yasemin = pd.read_csv("/home/musasina/projects/teknofest/msnet/datasets/yasemin_data.csv")
df_yasemin_notr = df_yasemin[df_yasemin["label"]==1].map(str)
df_yasemin_neg = df_yasemin[df_yasemin["label"]==2].map(str)
df_yasemin_pos = df_yasemin[df_yasemin["label"]==3].map(str)
df_yasemin_notr[df_yasemin_notr["label"]==1] = "Notr"
df_yasemin_neg[df_yasemin_neg["label"]==2] = "Negative"
df_yasemin_pos[df_yasemin_pos["label"]==3] = "Positive"
df_emre = pd.read_csv("/home/musasina/projects/teknofest/msnet/datasets/emre_data.csv")
df_emre_notr = df_emre[df_emre["label"]==1].map(str)
df_emre_neg = df_emre[df_emre["label"]==2].map(str)
df_emre_pos = df_emre[df_emre["label"]==3].map(str)
df_emre_notr[df_emre_notr["label"]==1] = "Notr"
df_emre_neg[df_emre_neg["label"]==2] = "Negative"
df_emre_pos[df_emre_pos["label"]==3] = "Positive"

In [6]:
splits = {'train': 'train.csv', 'test': 'test.csv'}
df = pd.read_csv("hf://datasets/winvoker/turkish-sentiment-analysis-dataset/" + splits["train"])
df_2 = pd.read_csv("hf://datasets/winvoker/turkish-sentiment-analysis-dataset/" + splits["test"])
df.drop("dataset",axis=1)
df_2.drop("dataset",axis=1)
df_train_notr = pd.concat((df[df["label"]=="Notr"],df_2[df_2["label"]=="Notr"],df_yasemin_notr,df_emre_notr),ignore_index=True)["text"].convert_dtypes(convert_string=True).to_list()
df_train_pos = pd.concat((df[df["label"]=="Positive"],df_2[df_2["label"]=="Positive"],df_yasemin_pos,df_emre_pos),ignore_index=True)["text"].convert_dtypes(convert_string=True).to_list()
df_train_neg = pd.concat((df[df["label"]=="Negative"],df_2[df_2["label"]=="Negative"],df_yasemin_neg,df_emre_neg),ignore_index=True)["text"].convert_dtypes(convert_string=True).to_list()

In [7]:
notr_bow = pd.read_csv("/home/musasina/projects/teknofest/msnet/bows/notr_bow_not_cleared.csv")
positive_bow = pd.read_csv("/home/musasina/projects/teknofest/msnet/bows/positive_bow_not_cleared.csv")
negative_bow = pd.read_csv("/home/musasina/projects/teknofest/msnet/bows/negative_bow_not_cleared.csv")

In [8]:
notr_bow_set = set(notr_bow["words"].map(str).to_list())
positive_bow_set = set(positive_bow["words"].map(str).to_list())
negative_bow_set = set(negative_bow["words"].map(str).to_list())

In [9]:
wrong_bow_set = notr_bow_set.intersection(positive_bow_set,negative_bow)
wrong_bow_set_pos_notr = notr_bow_set.intersection(positive_bow_set)
wrong_bow_set_pos_neg = negative_bow_set.intersection(positive_bow_set)
wrong_bow_set_neg_notr = negative_bow_set.intersection(notr_bow_set)

In [10]:
print(len(wrong_bow_set_pos_notr))
print(len(wrong_bow_set_pos_neg))
print(len(wrong_bow_set_neg_notr))

34771
93053
30050


In [11]:
random.seed(42)

In [12]:
notr_bow_set -= wrong_bow_set
positive_bow_set -= wrong_bow_set
negative_bow_set -= wrong_bow_set
#----------------------------------
#positive_bow_set -= set(random.sample(list(wrong_bow_set_pos_neg),len(wrong_bow_set_pos_neg)//2))
#----------------------------------

In [13]:
print(len(notr_bow_set))
print(len(positive_bow_set))
print(len(negative_bow_set))

58347
273831
186740


In [14]:
from itertools import repeat

In [15]:
cities = pd.read_csv("/home/musasina/projects/teknofest/msnet/datasets/cities.csv")["names"].to_list()
counties = pd.read_csv("/home/musasina/projects/teknofest/msnet/datasets/counties.csv")["names"].to_list()
names = pd.read_csv("/home/musasina/projects/teknofest/msnet/datasets/names.csv")["names"].to_list()
surnames = pd.read_csv("/home/musasina/projects/teknofest/msnet/datasets/surnames.csv")["names"].to_list()
countries = pd.read_csv("/home/musasina/projects/teknofest/msnet/datasets/countries.csv")["names"].to_list()
places = pd.read_csv("/home/musasina/projects/teknofest/msnet/datasets/palces.csv")["names"].to_list()

In [16]:
from nltk.corpus import stopwords
import re
import nltk

nltk.download("stopwords")
stops = set(stopwords.words("turkish"))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/musasina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
texts_with_companies = []
useless_pos = []
useless_neg = []
useless_notr = []

In [18]:
def add_place_to_text(truncated_text,labels,label):
    index_company = np.random.randint(0,len(truncated_text))
    index_city = np.random.randint(0,len(truncated_text))
    index_name = np.random.randint(0,len(truncated_text))
    index_surname = np.random.randint(0,len(truncated_text))
    index_place = np.random.randint(0,len(truncated_text))
    index_county = np.random.randint(0,len(truncated_text))
    index_country = np.random.randint(0,len(truncated_text))
    #----------------------------------------------------------
    company = str(np.random.choice(companies_array)).casefold().strip()
    city = np.random.choice(cities).casefold().strip()
    name = np.random.choice(names).casefold().strip()
    surname = np.random.choice(surnames).casefold().strip()
    place = np.random.choice(places).casefold().strip()
    county = np.random.choice(counties).casefold().strip()
    country = np.random.choice(countries).casefold().strip()
    #----------------------------------------------------------
    labels.insert(index_company,label)
    truncated_text.insert(index_company,company)
    #-----------------------------------------
    labels.insert(index_city,label)
    truncated_text.insert(index_city,city)
    #-----------------------------------------
    labels.insert(index_name,label)
    truncated_text.insert(index_name,name)
    #-----------------------------------------
    labels.insert(index_surname,label)
    truncated_text.insert(index_surname,surname)
    #-----------------------------------------
    labels.insert(index_place,label)
    truncated_text.insert(index_place,place)
    #-----------------------------------------
    labels.insert(index_county,label)
    truncated_text.insert(index_county,county)
    #-----------------------------------------
    labels.insert(index_country,label)
    truncated_text.insert(index_country,country)
    return truncated_text,labels

In [19]:
def add_place_to_text_2(truncated_text,labels,label):
    index_company = np.random.randint(0,len(truncated_text))
    index_city = np.random.randint(0,len(truncated_text))
    index_name = np.random.randint(0,len(truncated_text))
    index_surname = np.random.randint(0,len(truncated_text))
    index_place = np.random.randint(0,len(truncated_text))
    index_county = np.random.randint(0,len(truncated_text))
    index_country = np.random.randint(0,len(truncated_text))
    #----------------------------------------------------------
    company = str(np.random.choice(companies_array)).casefold().strip()
    city = np.random.choice(cities).casefold().strip()
    name = np.random.choice(names).casefold().strip()
    surname = np.random.choice(surnames).casefold().strip()
    place = np.random.choice(places).casefold().strip()
    county = np.random.choice(counties).casefold().strip()
    country = np.random.choice(countries).casefold().strip()
    #---------------------------------------------------------
    index = np.random.choice([index_company,index_city,index_name,index_surname,index_place,index_county,index_country])
    named_entity = np.random.choice([company,city,name,surname,place,county,country])
    #----------------------------------------------------------
    labels.insert(index,label)
    truncated_text.insert(index,named_entity)
    #-----------------------------------------
    return truncated_text,labels

In [20]:
def create_data(seperated_texts:list,label,add_palces=True):
    for truncated_text in seperated_texts:
        labels = [0]*25
        for i,word in enumerate(truncated_text):
            if label == 1:
                if word in notr_bow_set:
                    labels[i]=1
                if word in wrong_bow_set_pos_neg:
                    labels[i]=1
            elif label == 2:
                if word in negative_bow_set:
                    labels[i]=2
            elif label ==3:
                if word in positive_bow_set:
                    labels[i]=3
        if add_palces:
            yield add_place_to_text(truncated_text,labels,label)
        else:
            yield truncated_text,labels

In [21]:
def create_data_loop(data,sentiment):
    useless = []
    for text,label in zip(data,repeat(sentiment)):
        left = 0
        right = 25
        seperated_texts = []
        splitted_text = text.split()
        count = 0
        while left<right:
            truncated_text = splitted_text[left:right]
            if len(truncated_text) <25:
                if count == 0:
                    useless.append(" ".join(truncated_text))
                break
            count += 1
            seperated_texts.append(truncated_text)
            left = deepcopy(right)
            right += 25
        for truncated_text,labels in create_data(seperated_texts,label):
            texts_with_companies.append({
                "text":" ".join(truncated_text),
                "sentiment":label,
                "labels":labels
            })
    return useless

In [22]:
label_dict = {"Notr":1,"Negative":2,"Positive":3}

In [23]:
df_useless_notr = create_data_loop(df_train_notr,1)
df_useless_neg = create_data_loop(df_train_neg,2)
df_useless_pos = create_data_loop(df_train_pos,3)

In [24]:
print(len(df_useless_notr))
print(len(df_useless_neg))
print(len(df_useless_pos))

179812
39211
206857


In [25]:
df_useless = pd.DataFrame({"text":df_useless_notr+df_useless_neg+df_useless_pos,"label":[1]*len(df_useless_notr)+[2]*len(df_useless_neg)+[3]*len(df_useless_pos)})

In [26]:
df_useless_notr = df_useless[df_useless["label"]==1].sample(39211,random_state=42)
df_useless_neg = df_useless[df_useless["label"]==2].sample(39211,random_state=42)
df_useless_pos = df_useless[df_useless["label"]==3].sample(39211,random_state=42)

In [27]:
print(len(df_useless_notr))
print(len(df_useless_neg))
print(len(df_useless_pos))

39211
39211
39211


In [28]:
df_useless = pd.concat((df_useless_notr,df_useless_neg,df_useless_pos),ignore_index=True)

In [29]:
df_useless = sklearn.utils.shuffle(df_useless,random_state=42)

In [30]:
for _ in range(15):
    df_useless = sklearn.utils.shuffle(df_useless,random_state=42)
    concated_text = []
    labels = []
    for text,label in zip(df_useless["text"],df_useless["label"]):
        splitted_text = str(text).split()
        if len(splitted_text) != 0:
            for truncated_text,return_labels in create_data([splitted_text],label,add_palces=False):
                concated_text += truncated_text[:len(splitted_text)]
                labels += return_labels[:len(splitted_text)]
                concated_text,labels = add_place_to_text_2(concated_text,labels,label)
            if len(concated_text) < 32:
                continue
            concated_text = concated_text[:32]
            labels = labels[:32]
            notr_count = labels.count(1)
            neg_count = labels.count(2)
            pos_count = labels.count(3)
            overall_label = 0
            if notr_count == np.max([notr_count,neg_count,pos_count]):
                overall_label = 1
            elif neg_count == np.max([notr_count,neg_count,pos_count]):
                overall_label = 2
            else:
                overall_label = 3
            texts_with_companies.append({
            "text":" ".join(concated_text),
            "sentiment":overall_label,
            "labels":deepcopy(labels)
            })
            concated_text = []
            labels = []

In [31]:
import sklearn.utils


df_final = sklearn.utils.shuffle(pd.DataFrame(texts_with_companies),random_state=42)
df_final.to_csv("/home/musasina/projects/teknofest/msnet/datasets/train_final_1.5.csv",index=False)